In [1]:
#Importacion de librerias
import bempp.api
import numpy as np 
import time
from readoff import *
from readpqr import *
start = time.time()

In [2]:
#Datos Principales. 
em = 4.     #[-] Permitividad electrica interior.
es = 80.    #[-] Permitividad electrica exterior.
k = 0.125   #[1/A] Inverso de la longitud de Debey-Huckel del fluido.

#Eleccion de resolver la ecuacion matricial en BEM/BEM
SF = True  #Strong form: True utiliza el precondicionador Mass Matrix. False no se utiliza ningun precondicionador.

#Ensablaje de los operadores de frontera: #'fmm' para moleculas con un gran mayor numero de vertices.
#'default_nonlocal' para moleculas con un pequeño numero de vertices.
Assemble = 'default_nonlocal' 

In [3]:
#Datos de la posicion en el espacio, carga y radios de los atomos de la molecula.
PC,Q,R = readpqr('PQR/Sphere5Q3.pqr')  #Eleccion del "pqr" de la molecula

In [4]:
#Generar la malla superficial de la molecula
#En caso que la malla tenga huecos pequeños, con trimesh se obtiene la informacion de la malla original sin los huecos.
import trimesh
V1,F1 = read_off('Mallas_S/Sphere5R0.off') #Eleccion del nombre de la malla en formato "off"
meshSP = trimesh.Trimesh(vertices = V1, faces= F1) 
mesh_split = meshSP.split()
print("Found %i meshes"%len(mesh_split))

vertices_split1 = mesh_split[0].vertices 
faces_split1 = mesh_split[0].faces   
grid = bempp.api.grid.grid.Grid(vertices_split1.transpose(), faces_split1.transpose()) #Creacion de la malla superficial

Found 1 meshes


In [5]:
#Generar espacios de funcionales del potencial y su derivada
dirichl_space = bempp.api.function_space(grid, "P", 1)   #Potencial electrostatico en la interfaz.
neumann_space = bempp.api.function_space(grid, "DP", 0)  #Derivada del potencial electrostatico en la interfaz.

print("DS dofs: {0}".format(dirichl_space.global_dof_count))
print("NS dofs: {0}".format(neumann_space.global_dof_count))

DS dofs: 290
NS dofs: 576


In [6]:
#Generar los operadores de frontera
bempp.api.GLOBAL_PARAMETERS.fmm.expansion_order = 5
#Dominio del soluto Ωm
IL = bempp.api.operators.boundary.sparse.identity(dirichl_space, dirichl_space, dirichl_space) #1
KL = bempp.api.operators.boundary.laplace.double_layer(dirichl_space, dirichl_space, dirichl_space, assembler=Assemble) #K
VL = bempp.api.operators.boundary.laplace.single_layer(neumann_space, dirichl_space, dirichl_space, assembler=Assemble) #V
#Dominio del solvente Ωs
IH = bempp.api.operators.boundary.sparse.identity(dirichl_space, neumann_space, neumann_space) #1
if k==0:
    KH = bempp.api.operators.boundary.laplace.double_layer(dirichl_space, neumann_space, neumann_space, assembler=Assemble) #K
    VH = bempp.api.operators.boundary.laplace.single_layer(neumann_space, neumann_space, neumann_space, assembler=Assemble) #V
else:
    KH = bempp.api.operators.boundary.modified_helmholtz.double_layer(dirichl_space, neumann_space, neumann_space, k, assembler=Assemble) #K
    VH = bempp.api.operators.boundary.modified_helmholtz.single_layer(neumann_space, neumann_space, neumann_space, k, assembler=Assemble) #V

In [7]:
#Creacion de funcion del potencial de Coulomb    
@bempp.api.complex_callable(jit=False)
def U_c(x, n, domain_index, result):
    global Q,PC,em
    result[:] = (1 / (4.*np.pi*em))  * np.sum( Q / np.linalg.norm( x - PC, axis=1))
U_c = bempp.api.GridFunction(dirichl_space, fun=U_c)

#Construccion del vector derecho
if SF==False:
    # Rhs en Ωm
    rhs_M = (U_c).projections(dirichl_space)
    # Rhs en Ωs
    rhs_S = np.zeros(neumann_space.global_dof_count)
    # La conbinacion del rhs
    rhs = np.concatenate([rhs_M, rhs_S])
else:    
    rhs = [IL*U_c, 0*IH*U_c]

/home/mauricioguerrero/Software/miniconda3/envs/bempp_prod/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


In [8]:
#Construccion matriz izquierda 2x2
if SF==False:
    #Posicion de la matriz 2x2
    blocks = [[None,None],[None,None]] 
    blocks[0][0] = (0.5*IL + KL).weak_form()  #0.5+K
    blocks[0][1] = -VL.weak_form()            #-V
    blocks[1][0] = (0.5*IH - KH).weak_form()  #0.5-K
    blocks[1][1] = (em/es)*VH.weak_form()     #V(em/es)    
    blocked = bempp.api.assembly.blocked_operator.BlockedDiscreteOperator(np.array(blocks))
    
    #Precondicionador block diagonal opcional para BEM.
    from preconditioners import *
    #P = BlockDiagonal0(dirichl_space, neumann_space, blocks, es, em, k) 
else:   
    #Posicion de la matriz 2x2
    blocks = bempp.api.BlockedOperator(2,2)    
    blocks[0,0] = (0.5*IL + KL)  #0.5+K
    blocks[0,1] = -VL            #-V
    blocks[1,0] = (0.5*IH - KH)  #0.5-K
    blocks[1,1] = VH*(em/es)     #V(em/es)

In [9]:
#Se resuelve la solucion de la Ecuacion matricial Ax=B
#Contador de iteraciones
it_count = 0
def count_iterations(x):
    global it_count
    it_count += 1
    if (it_count / 1000) == (it_count // 1000):
        print(it_count,x)

# Solucion por GMRES
from scipy.sparse.linalg import gmres
if SF==False:
    start1 = time.time()
    soln, info = gmres(blocked, rhs, M=None, callback=count_iterations,tol=1e-6)  #Sin Precondicionador#
    end1 = time.time() 
else:
    #bempp.api.enable_console_logging('info')
    start1 = time.time()
    soln, info, res, it_count = bempp.api.linalg.gmres(blocks, rhs, return_residuals=True, return_iteration_count=True, use_strong_form=True, tol=1e-6)
    end1 = time.time()

# Tiempo de resolver la ecuacion
curr_time1 = (end1 - start1) 
print("Numero de iteraciones de GMRES: {0}".format(it_count))
print("Tiempo total en GMRES: {:5.2f} [s]".format(curr_time1))

/home/mauricioguerrero/Software/miniconda3/envs/bempp_prod/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


Numero de iteraciones de GMRES: 136
Tiempo total en GMRES:  2.30 [s]


In [10]:
#Calcula todo el dominio global del potencial a partir de la solucion del borde calculado
if SF==False:
    # Solucion para datos de Dirichlet 
    soln_u = soln[:dirichl_space.global_dof_count]
    dirichlet_fun = bempp.api.GridFunction(dirichl_space, coefficients=soln_u)
    # Solucion para datos de Neumann 
    soln_du = soln[dirichl_space.global_dof_count:] 
    neumann_fun = bempp.api.GridFunction(neumann_space, coefficients=soln_du)
else:
    dirichlet_fun = soln[0]
    neumann_fun = soln[1]

In [11]:
#Calculo del potencial en la posicion de los atomos de la molecula
VF = bempp.api.operators.potential.laplace.single_layer(neumann_space, np.transpose(PC)) 
KF = bempp.api.operators.potential.laplace.double_layer(dirichl_space, np.transpose(PC))
uF = VF*neumann_fun - KF*dirichlet_fun

#Resultado de la energia de solvatacion por atomo
for i in range(len(PC)):
    Sum1 = (uF[0][i]).real*Q[i]
    Ei = 0.5*4.*np.pi*332.064*(Sum1)
    print(i,Ei)

#Resultado de la energia de solvatacion total
E_Solv = 0.5*4.*np.pi*332.064*np.sum(Q*uF).real 
print('Energia de Solvatacion: {:7.6f} [kCal/mol]'.format(E_Solv) )

end = time.time()
curr_time = (end - start)   
print("Tiempo total: {:5.2f} [s]".format(curr_time))

0 -24.300606862083175
1 -24.30250296328726
2 -24.301953286568892
Energia de Solvatacion: -72.905063 [kCal/mol]
Tiempo total: 23.52 [s]
